# Auto Generated Agent Chat: Using RetrieveChat for Retrieve Augmented Code Generation and Question Answering

RetrieveChat is a convesational framework for retrieve augmented code generation and question answering. In this notebook, we demonstrate how to utilize RetrieveChat to generate code and answer questions based on customized documentations that are not present in the LLM's training dataset. RetrieveChat uses the `RetrieveAssistantAgent` and `RetrieveUserProxyAgent`, which is similar to the usage of `AssistantAgent` and `UserProxyAgent` in other notebooks (e.g., [Automated Task Solving with Code Generation, Execution & Debugging](https://github.com/microsoft/FLAML/blob/main/notebook/autogen_agentchat_auto_feedback_from_code_execution.ipynb)). Essentially,`RetrieveAssistantAgent` and  `RetrieveUserProxyAgent` implements a different auto reply mechanism corresponding to the RetrieveChat prompts.

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [mathchat] option.
```bash
pip install flaml[retrievechat]
```

In [2]:
# %pip install flaml[retrievechat]~=2.0.0rc5

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [3]:
from flaml import autogen

config_list = autogen.config_list_from_json(
    env_or_file=".config.local",
    file_location=".",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-35-turbo",
            "gpt-3.5-turbo",
        }
    },
)

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-4']


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 and gpt-3.5-turbo models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct agents for RetrieveChat

We start by initialzing the `RetrieveAssistantAgent` and `RetrieveUserProxyAgent`. The system message needs to be set to "You are a helpful assistant." for RetrieveAssistantAgent. The detailed instructions are given in the user message. Later we will use the `RetrieveUserProxyAgent.generate_init_prompt` to combine the instructions and a math problem for an initial prompt to be sent to the LLM assistant.

In [4]:
from flaml.autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from flaml.autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

autogen.ChatCompletion.start_logging()

# 1. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant", 
    system_message="You are a helpful assistant.",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
)

# 2. create the RetrieveUserProxyAgent instance named "ragproxyagent"
# By default, the human_input_mode is "ALWAYS", which means the agent will ask for human input at every step. We set it to "NEVER" here.
# `docs_path` is the path to the docs directory. By default, it is set to "./docs". Here we generated the documentations from FLAML's docstrings.
# Navigate to the website folder and run `pydoc-markdown` and it will generate folder `reference` under `website/docs`.
# `chunk_token_size` is the chunk token size for the retrieve chat. By default, it is set to `max_tokens * 0.6`, here we set it to 2000.
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "docs_path": "../website/docs/reference",
        "chunk_token_size": 2000,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "embedding_model": "all-mpnet-base-v2",
    },
)

### Example 1

Use RetrieveChat to help generate sample code and automatically run the code and fix errors if there is any.

Problem: Which API should I use if I want to use FLAML for a classification task and I want to train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.

In [5]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# given a problem, we use the ragproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the ragproxyagent for processing.
# The conversation continues until the termination condition is met, in RetrieveChat, the termination condition when no human-in-loop is no code block detected.
# With human-in-loop, the conversation will continue until the user says "exit".
code_problem = "How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached."
ragproxyagent.initiate_chat(assistant, problem=code_problem, search_string="spark")  # search_string is used as an extra filter for the embeddings search, in this case, we only want to search documents that contain "spark".

ERROR:flaml.autogen.retrieve_utils:Collection flaml-docs already exists.


doc_ids:  [['doc_35', 'doc_40', 'doc_14', 'doc_21', 'doc_15', 'doc_51', 'doc_44', 'doc_52', 'doc_41', 'doc_45', 'doc_13', 'doc_61', 'doc_38', 'doc_36', 'doc_8']]
Adding doc_id doc_35 to context.
Adding doc_id doc_40 to context.
Adding doc_id doc_14 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
You should leverage the context provided by the user as much as possible. If you need more context, you should reply
"UPDATE CONTEXT".
For code generation task, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats belo

your 131072x1 screen size is bogus. expect trouble


23/08/10 10:40:56 WARN Utils: Your hostname, DESKTOP-TTKT4BA resolves to a loopback address: 127.0.1.1; using 172.30.239.248 instead (on interface eth0)
23/08/10 10:40:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


...

[flaml.tune.tune: 08-10 10:41:22] {749} INFO - Brief result: {'pred_time': 4.839897155761719e-05, 'wall_clock_time': 28.937474250793457, 'metric_for_logging': {'pred_time': 4.839897155761719e-05}, 'val_loss': 0.18647493060299933, 'trained_estimator': <flaml.automl.model.CatBoostEstimator object at 0x7f70a91af9a0>}
[flaml.tune.tune: 08-10 10:41:22] {729} INFO - Number of trials: 4/1000000, 1 RUNNING, 3 TERMINATED


/home/lijiang1/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.tune.tune: 08-10 10:41:23] {749} INFO - Brief result: {'pred_time': 5.043943723042806e-05, 'wall_clock_time': 29.686672925949097, 'metric_for_logging': {'pred_time': 5.043943723042806e-05}, 'val_loss': 0.7215296790003777, 'trained_estimator': <flaml.automl.model.XGBoostSklearnEstimator object at 0x7f70a88e35e0>}
[flaml.tune.tune: 08-10 10:41:23] {729} INFO - Number of trials: 5/1000000, 1 RUNNING, 4 TERMINATED


/home/lijiang1/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/lijiang1/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/lijiang1/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/lijiang1/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
Time exceeded, canceled jobs                                                    



[flaml.automl.logger: 08-10 10:41:24] {2493} INFO - selected model: None
[flaml.automl.logger: 08-10 10:41:24] {2627} INFO - retrain catboost for 0.4s
[flaml.automl.logger: 08-10 10:41:24] {2630} INFO - retrained model: <catboost.core.CatBoostClassifier object at 0x7f70a91afdc0>
[flaml.automl.logger: 08-10 10:41:24] {1930} INFO - fit succeeded
[flaml.automl.logger: 08-10 10:41:24] {1931} INFO - Time taken to find the best model: 28.937474250793457
Best model: catboost
Best model's config: {'early_stopping_rounds': 10, 'learning_rate': 0.09999999999999996, 'n_estimators': 47}

>>>>>>>> EXECUTING CODE BLOCK 1 (inferred language is bash)...
ragproxyagent (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
None
You MUST NOT install any packages because all the packages needed are already installed.

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

TERMINATE

--------------------------------------------------------

### Example 2

Use RetrieveChat to answer a question that is not related to code generation.

Problem: Who is the author of FLAML?

In [6]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "Who is the author of FLAML?"
ragproxyagent.initiate_chat(assistant, problem=qa_problem)

doc_ids:  [['doc_58', 'doc_51', 'doc_35', 'doc_3', 'doc_22', 'doc_40', 'doc_14', 'doc_13', 'doc_59', 'doc_52', 'doc_1', 'doc_6', 'doc_28', 'doc_56', 'doc_29', 'doc_2', 'doc_55', 'doc_44', 'doc_19', 'doc_32']]
Adding doc_id doc_58 to context.
Adding doc_id doc_51 to context.
Adding doc_id doc_51 to context.
Adding doc_id doc_35 to context.
Adding doc_id doc_3 to context.
Adding doc_id doc_22 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
You should leverage the context provided by the user as much as possible. If you need more context, you should reply
"UPDATE CONTEXT".
For code generation task, you must obey the following

### Example 3

Use RetrieveChat to help generate sample code and ask for human-in-loop feedbacks.

Problem: how to build a time series forecasting model for stock price using FLAML?

In [7]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
code_problem = "how to build a time series forecasting model for stock price using FLAML?"
ragproxyagent.initiate_chat(assistant, problem=code_problem)

doc_ids:  [['doc_37', 'doc_35', 'doc_46', 'doc_40', 'doc_49', 'doc_51', 'doc_58', 'doc_48', 'doc_14', 'doc_39', 'doc_47', 'doc_41', 'doc_13', 'doc_60', 'doc_52', 'doc_59', 'doc_43', 'doc_10', 'doc_34', 'doc_33']]
Adding doc_id doc_37 to context.
Adding doc_id doc_35 to context.
Adding doc_id doc_46 to context.
Adding doc_id doc_35 to context.
Adding doc_id doc_46 to context.
Adding doc_id doc_40 to context.
Adding doc_id doc_49 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
You should leverage the context provided by the user as much as possible. If you need more context, you should reply
"UPDATE CONTEXT".
For code genera

### Example 4

Use RetrieveChat to answer a question and ask for human-in-loop feedbacks.

Problem: Is there a function named `tune_automl` in FLAML?

In [8]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
qa_problem = "Is there a function named `tune_automl` in FLAML?"
ragproxyagent.initiate_chat(assistant, problem=qa_problem)

doc_ids:  [['doc_35', 'doc_14', 'doc_40', 'doc_13', 'doc_51', 'doc_58', 'doc_20', 'doc_26', 'doc_34', 'doc_22', 'doc_11', 'doc_59', 'doc_3', 'doc_56', 'doc_47', 'doc_53', 'doc_19', 'doc_28', 'doc_32', 'doc_46']]
Adding doc_id doc_35 to context.
Adding doc_id doc_14 to context.
Adding doc_id doc_40 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
You should leverage the context provided by the user as much as possible. If you need more context, you should reply
"UPDATE CONTEXT".
For code generation task, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already i

### Example 5

Use RetrieveChat to answer questions for [NaturalQuestion](https://ai.google.com/research/NaturalQuestions) dataset.

We'll first create a new document collection based on all the context corpus, then we select some questions and answer them with RetrieveChat.


In [5]:
corpus_file = "https://huggingface.co/datasets/thinkall/NaturalQuestionsQA/resolve/main/corpus.txt"

# Create a new collection for NaturalQuestions dataset
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "docs_path": corpus_file,
        "chunk_token_size": 4900,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "natural-questions",
        "chunk_mode": "one_line",
        "embedding_model": "all-MiniLM-L6-v2",
    },
)

In [6]:
import json

# queries_file = "https://huggingface.co/datasets/thinkall/NaturalQuestionsQA/resolve/main/queries.jsonl"
first_5_queries = """{"_id": "ce2342e1feb4e119cb273c05356b33309d38fa132a1cbeac2368a337e38419b8", "text": "what is non controlling interest on balance sheet", "metadata": {"answer": ["the portion of a subsidiary corporation 's stock that is not owned by the parent corporation"]}}
{"_id": "3a10ff0e520530c0aa33b2c7e8d989d78a8cd5d699201fc4b13d3845010994ee", "text": "how many episodes are in chicago fire season 4", "metadata": {"answer": ["23"]}}
{"_id": "8a19db28aa9be7dbcab290077d4a3e320d87e8badce1f7417f56f3720e527696", "text": "who sings love will keep us alive by the eagles", "metadata": {"answer": ["Timothy B. Schmit"]}}
{"_id": "06758a4918650a3f27398ff0520d4121c6d9cacfab3efcd84ddb83d2caa460ae", "text": "who is the leader of the ontario pc party", "metadata": {"answer": ["Patrick Walter Brown"]}}
{"_id": "d4a88ef2f705da63ced53932695505a2fc44879bafe6fd1224ffef9e9a590ad6", "text": "where did the last name keith come from", "metadata": {"answer": ["from Keith in East Lothian , Scotland", "from a nickname , derived from the Middle High German k\u012bt , a word meaning `` sprout '' , `` offspring ''"]}}
"""
queries = [json.loads(line) for line in first_5_queries.split("\n") if line]
questions = [q["text"] for q in queries]
answers = [q["metadata"]["answer"] for q in queries]
print(questions)
print(answers)

['what is non controlling interest on balance sheet', 'how many episodes are in chicago fire season 4', 'who sings love will keep us alive by the eagles', 'who is the leader of the ontario pc party', 'where did the last name keith come from']
[["the portion of a subsidiary corporation 's stock that is not owned by the parent corporation"], ['23'], ['Timothy B. Schmit'], ['Patrick Walter Brown'], ['from Keith in East Lothian , Scotland', "from a nickname , derived from the Middle High German kīt , a word meaning `` sprout '' , `` offspring ''"]]


In [9]:
for i in range(2):
    print(f"\n\n>>>>>>>>>>>>  Below are outputs of Case {i+1}  <<<<<<<<<<<<\n\n")

    # reset the assistant. Always reset the assistant before starting a new conversation.
    assistant.reset()

    # set `human_input_mode` to be `NEVER`
    ragproxyagent.human_input_mode = "NEVER"
    qa_problem = questions[i]
    ragproxyagent.initiate_chat(assistant, problem=qa_problem, n_results=3)



>>>>>>>>>>>>  Below are outputs of Case 1  <<<<<<<<<<<<


doc_ids:  [['doc_0', 'doc_155', 'doc_19']]
Adding doc_id doc_0 to context.
Adding doc_id doc_155 to context.
Adding doc_id doc_155 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
You should leverage the context provided by the user as much as possible. If you need more context, you should reply
"UPDATE CONTEXT".
For code generation task, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

For que